### Multi-layer Perceptrons

Softmax Regression can be classified into multi-classification method. It can be considered a simple Neural Network without hidden layer.

Hidden layer enables the Neural Network with more functional features to fit more complicated and non-linear model. Theorically, the number of the hidden nodes are exponentially decreasing with the increase of the hidden layers. In other words, the more hidden layers, the less hidden layers required. 

In real problems, many troubles could be encounted with the deeper layer in a network, such as: Overfitting, Parameters-tunning, or Gradient Vanishment. Recently, more and more tricks have came out to deal with those problems, such as Dropout, Adagrad, ReLU...

1. Overfit can be common in machine learning.Dropout was proposed by Hinton. The idea is to drop a partial output nodes randomly when training the model.This method is equal to create more random samples, hence the overfit can be provented by enlarging the samples set.<br>
Dropout can also be considered a bagging method, and it resample the feeatures by dropping the nodes.

2. Parameters are difficult to be tuned. Neural Networks are ususlly not convex problem, so the results can be trapped around the local optimal. The parameters such as Learning Rate can be really matters when training with SGD algorithm, and the results could be totally different when different learning rate applied.<br>
More adaptive methods came out to deal with this trouble. For more info, please see my another respo: https://github.com/vince-xunzhe/Convex-Optimizer-in-Deep-Learning

3. Gradient Vanishment is not expected in deep learning. Before ReLU, Sigmoid served as the activated function for many Neural Network applications. This might result from the probabilities output from 0 to 1. But the increment is large in central area, small at the sides. It is like the biological Neuron model. But the problems is the gradient of Sigmoid can be vanished with network grows deeper, which cannot train the model efficiently.<br>
ReLU solved this problem,as: $$y=max(0,x)$$ and its gradient cannot be vanished after many layers of back-prop.<br>
ReLU mimiced the human brain model, and fixed the Gradient Vanishment. From now, we don't need another unsupervised learning layer by layer to initialize the weights.<br>
In many real cases, the training speed and accuracy could benefit from replacing Sigmoid with ReLU in hidden layer, but the output layer remained to use the Sigmoid since it represents the probabilistic distribution.  

Above, I mentioned Dropout, Adagrad, ReLU... to deal with the multi-layer neural network problems. So what significant benefits we can derive from the deeper network? I can say with the more hidden layers of the neural network, the more abstract the original features can be transformed, and the stronger the model's ability to fit.

Next, I am trying to add a hidden layer to our previously written Neural Network.

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)
sess = tf.InteractiveSession()

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [5]:
in_units = 784
h1_units = 300
w1 = tf.Variable(tf.truncated_normal([in_units, h1_units], stddev=0.1))
b1 = tf.Variable(tf.zeros([h1_units]))
w2 = tf.Variable(tf.zeros([h1_units, 10]))
b2 = tf.Variable(tf.zeros([10]))

x = tf.placeholder(tf.float32,[None, in_units])
keep_prob = tf.placeholder(tf.float32) # the ratio of Dropout, the ratio differs in training & prediction

The hidden layer formula: $$y=relu(W_1x+b_1)$$
the model construction can be defined as:

In [7]:
hidden1 = tf.nn.relu(tf.matmul(x, w1) + b1)
hidden1_drop = tf.nn.dropout(hidden1, keep_prob)
y = tf.nn.softmax(tf.matmul(hidden1_drop, w2) + b2)

So far the forward computation of the Neural Network were constructed. Then, I will define the loss function & optimizer. (Cross-entropy & Adagrad).

In [8]:
y_ = tf.placeholder(tf.float32, [None, 10])
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))
train_step = tf.train.AdagradOptimizer(0.3).minimize(cross_entropy)

Training processing.

In [9]:
tf.global_variables_initializer().run()

for i in range(3000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    train_step.run({x: batch_xs, y_: batch_ys, keep_prob: 0.75})

Evaluation of the model accuracy.

In [10]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(accuracy.eval({x: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}))

0.9802


MLP is also known as Fully Connected Network(FCN). It also has limitations, even though we used deeper networks, more hidden nodes, or more epochs, it's hard to get a better accuracy.